# Aprendizado de Máquina Aplicada à Análise de Dados do COVID-19

- Professor: Filipe Verri
- Aluno: Paulo Giovani de Faria Zeferino

Este primeiro notebook contém os passos realizados para o download e preparação dos dados utilizados para o desenvolvimento do artigo **Aprendizado de Máquina Aplicada à Análise de Dados do COVID-19**, solicitado pelo professor Filipe Verri para a disciplina PO-233/2020 Aprendizado de Máquina, oferecida pelo curso de pós-graduação do ITA. 


# 1. Download dos dados do repositório Johns Hopkins

Os dados são obtidos diretamento do repositório do [Johns Hopkins](https://github.com/CSSEGISandData/COVID-19), para que os valores estejam sempre atualizados. Observe que, diante disso, o número total de linhas do arquivo final que será gerado para aplicação das técnicas de Aprendizado de Máquina poderá sofrer variações.

In [1]:
# Importa as bibliotecas
import pandas as pd
import numpy as np
import wget
import ssl

In [7]:
# Cria o certificado para poder realizar o download
ssl._create_default_https_context = ssl._create_unverified_context

# Url do repositório RAW, com os datasets das séries temporais
data_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'

# Lista com os arquivos CSV dos datasets globais: casos confirmados e mortes
urls = [data_url + 'csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', 
        data_url + 'csse_covid_19_time_series/time_series_covid19_deaths_global.csv']

# Define os nomes para salvar os arquivos
nomes = ['confirmados.csv', 'mortes.csv']

# Realiza o download dos dados e salva no diretório 'data/atualizados/'
for i,url in enumerate(urls):
    nome = 'data/atualizados/' + nomes[i]
    filename = wget.download(url, out=nome)
    
# Os dados utilizados no artigo foram coletados no dia 16 de abril de 2010
# Eles estão armazenados no diretório 'data/artigo/'

# 2. Download dos dados de estatística adicionais

As informações sobre os dados alguns dados demográficos foi obtida em [Countries in the world by population (2020)](https://www.worldometers.info/world-population/population-by-country/). Para isso, criou-se um script de raspagem de dados, baseado nas informações disponíveis neste [kernel do Kaggle](https://www.kaggle.com/tanuprabhu/population-by-country-2020).

Informações como código dos países, taxa de fumantes, taxa de mortes por doenças pulmonares e total de leitos hospitalares, dentre outras, foram obtidas por meio do download de um conjunto de dados disponibilizado no Kaggle, pelo usuário [Korito](https://www.kaggle.com/koryto/countryinfo#covid19countryinfo.csv). Após a descompactação, o arquivo **covid19countryinfo.csv** foi renomeado para **dados_extras.csv** e salvo no diretório 'data/artigo/'.

In [8]:
# Importa as bibliotecas
import requests
from bs4 import BeautifulSoup

# Url para a tabela HTML que contém com os dados
url = "https://www.worldometers.info/world-population/population-by-country/"

# Escaneia o código HTML da url e carrega o conteúdo para dentro de uma lista do Pandas
r = requests.get(url)
soup = BeautifulSoup(r.content)
countries = soup.find_all("table")[0]
df = pd.read_html(str(countries))[0]

# Função para montar o DataFrame
def create_dataframe(a, b, c, d, e, f, g, h, i, j, k):
    data = pd.DataFrame(
        {
            'a': df[a],
            'b': df[b],
            'c': df[c],
            'd': df[d],
            'e': df[e],
            'f': df[f],
            'g': df[g],
            'h': df[h],
            'i': df[i],
            'j': df[j],
            'k': df[k]
        }
    )
    return data

# Define os nomes para as colunas do DataFrame
cols = ['Country (or dependency)', 'Population (2020)', 'Yearly Change', 'Net Change', 'Density (P/Km²)', 'Land Area (Km²)', 
        'Migrants (net)', 'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share']

# Monta o DataFrame, a partir dos dados HTML
df = create_dataframe(*cols)

# Renomeia as colunas do DataFrame
df.columns = [*cols]

# Exporta o arquivo CSV para dentro do diretório 'data/artigo/'
df.to_csv('data/artigo/dados_paises.csv', index=None, header=True)

# 3. Merge dos arquivos de dados: *confirmados.csv* e *mortes.csv*

Nessa etapa, unimos as colunas entre os arquivos com os dados globais de casos confirmados e de fatalidades.

In [10]:
# Carrega os dados dos arquivos CSV
confirmados_df = pd.read_csv('data/artigo/confirmados.csv')
mortes_df = pd.read_csv('data/artigo/mortes.csv')

In [11]:
# Realiza a união das colunas entre os arquivos com os dados globais dos casos confirmados e das fatalidades

# Recupera as colunas com as datas das medições
datas = confirmados_df.columns[4:]

# Gera um novo DataFrame, despivotando as colunas do DataFrame 'confirmados_df', transformando-as em linhas
confirmados_df_unpivot = confirmados_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                                             value_vars=datas, 
                                             var_name='Date', 
                                             value_name='Confirmados')

# Gera um novo DataFrame, despivotando as colunas do DataFrame 'mortes_df', transformando-as em linhas
mortes_df_unpivot = mortes_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                                   value_vars=datas, 
                                   var_name='Date', 
                                   value_name='Fatalidades')

# Gera um novo DataFrame, concatenando as colunas dos últimos DataFrames que foram gerados
temp1_df = pd.concat([confirmados_df_unpivot, mortes_df_unpivot['Fatalidades']], axis=1, sort=False)

In [12]:
# Remove algumas inconsistências existentes no último DataFrame -> 'temp1_df'

# Remove as linhas onde a coluna 'Province/State' = 'Recovered'
temp1_df = temp1_df.loc[temp1_df['Province/State'].str.contains('Recovered') != True]

# Renomeia os países, províncias e regiões - Coréia do Sul
temp1_df['Country/Region'] = temp1_df['Country/Region'].replace('Korea, South', 'South Korea')

# Renomeia os países, províncias e regiões - Taiwan
temp1_df['Country/Region'] = temp1_df['Country/Region'].replace('Taiwan*', 'Taiwan')

# Renomeia os países, províncias e regiões - Estados Unidos
temp1_df['Country/Region'] = temp1_df['Country/Region'].replace('US', 'United States')

# 4. Merge dos arquivos de dados: *temp1_df* e *dados_paises.csv*

Nessa etapa, unimos as colunas do DataFrame que contém os dados de casos globais confirmados e das fatalidades, junto com as colunas do arquivo que contém os dados dos países.

In [13]:
# Carrega o arquivo com os dados dos países
dados_paises_df = pd.read_csv('data/artigo/dados_paises.csv')

# Remove as colunas de 'dados_paises_df', que não serão utilizadas
dados_paises_df = dados_paises_df.drop(['Yearly Change', 'Net Change', 'Migrants (net)', 'World Share'], axis = 1) 

# Renomeia as colunas do DataFrame 'temp1_df'
temp1_df = temp1_df.rename(columns = {
    'Province/State': 'Província/Estado', 
    'Country/Region': 'País/Região',
    'Lat': 'Latitude',
    'Long': 'Longitude',
    'Date': 'Data',
    'Confirmados': 'Casos Confirmados'
})

# Renomeia as colunas do DataFrame 'dados_paises_df'
dados_paises_df = dados_paises_df.rename(columns = {
    'Country (or dependency)': 'País/Região',
    'Population (2020)': 'População',
    'Density (P/Km²)': 'Densidade (Km/2)',
    'Land Area (Km²)': 'Área (Km/2)',
    'Fert. Rate': 'Taxa de Fertilidade',
    'Med. Age': 'Média de Idade',
    'Urban Pop %': 'População Urbana'
})

# Gera um novo DataFrame, unindo as colunas dos DataFrames 'temp1_df' e 'dados_paises_df'                      
temp2_df = pd.merge(temp1_df, dados_paises_df, how ='left', on='País/Região')

# 5. Preparação do arquivo *dados_extras.csv*

Nessa etapa, preparamos o arquivo *dados_extras_csv*, removendo colunas que não serão utilizadas e inconsistências, além de alterar a ordem das colunas restantes, para poder realizar o merge junto ao DataFrame *temp2_df*.

In [14]:
# Carrega o arquivo com os dados extras
dados_extras_df = pd.read_csv('data/artigo/dados_extras.csv')

# Remove as colunas de 'dados_extras_df', que não serão utilizadas
# Algumas das informações dessas colunas já existem no DataFrame 'temp2_df' e outras não serão utilizadas neste projeto
# Algumas, inclusive, apresentam muitos valores nulos
dados_extras_df = dados_extras_df.drop(['region', 'alpha2code', 'pop', 'tests', 'testpop', 'density', 'medianage', 'urbanpop', 
                                        'quarantine', 'schools', 'publicplace', 'gatheringlimit', 'gathering', 'nonessential', 
                                        'sex0', 'sex14', 'sex25', 'sex54', 'sex64', 'sex65plus', 'sexratio', 'femalelung', 
                                        'malelung', 'healthexp', 'healthperpop', 'fertility','totalcases', 'active30', 
                                        'active31', 'active1', 'active2', 'active3', 'newcases30', 'newcases31', 'newcases1', 
                                        'newcases2', 'newcases3', 'deaths', 'newdeaths30', 'newdeaths31', 'newdeaths1', 
                                        'newdeaths2', 'newdeaths3', 'recovered', 'critical30', 'critical31', 'critical1', 
                                        'critical2', 'critical3', 'casediv1m', 'deathdiv1m'], axis = 1)

# Renomeia as as colunas de 'dados_extras_df', para poder realizar o merge junto ao DataFrame 'temp2_df'
dados_extras_df = dados_extras_df.rename(columns = {
    'country': 'País/Região',
    'alpha3code': 'Código do País',
    'hospibed': 'Total de Leitos Hospitalares',
    'smokers': 'Taxa de Fumantes',
    'lung': 'Taxa de Mortalidade por Doenças Pulmonares',
    'gdp2019': 'PIB 2019',
    'avgtemp': 'Temperatura Média (Janeiro - Março)',
    'avghumidity': 'Taxa de Umidade (Janeiro - Março)',
    'firstcase': 'Data do Primeiro Caso'
})

# Renomeia os países, províncias e regiões - Coréia do Sul
dados_extras_df['País/Região'] = dados_extras_df['País/Região'].replace('Korea, South', 'South Korea')

# Renomeia os países, províncias e regiões - Taiwan
dados_extras_df['País/Região'] = dados_extras_df['País/Região'].replace('Taiwan*', 'Taiwan')

# Renomeia os países, províncias e regiões - Estados Unidos
dados_extras_df['País/Região'] = dados_extras_df['País/Região'].replace('US', 'United States')

# Lista com o nome dos países
nomes_paises = sorted(list(dados_extras_df['País/Região'].unique()))

# Remove as linhas onde o Código do País é do tipo NaN
for nome in nomes_paises:
    # Recupera os índices das linhas que devem ser removidas
    pais_index = dados_extras_df[(dados_extras_df['País/Região'] == nome) & 
                                 (dados_extras_df['Código do País'].isnull())].index
    # Remove as linhas do DataFrame 'info_paises_df'
    dados_extras_df = dados_extras_df.drop(pais_index)

In [15]:
# Exibe uma listagem com os países que devem se tratados
# Essas informações apresentam duplicações, pois, já existem no DataFrame 'temp2_df'
paises = ['China', 'Denmark', 'France', 'Netherlands', 'United States']
cols = ['País/Região', 'Código do País', 'PIB 2019']

for pais in paises:
    print(dados_extras_df[dados_extras_df['País/Região'] == pais][cols])
    print()

    País/Região Código do País    PIB 2019
33        China            HKG     372,989
34        China            CHN  14,140,163
111       China            MAC      55,136

   País/Região Código do País PIB 2019
44     Denmark            DNK  347,176
57     Denmark            FRO      NaN
73     Denmark            GRL    2,441

    País/Região Código do País   PIB 2019
60       France            FRA  2,707,074
61       France            GUF     23,587
62       France            PYF      5,623
63       France            NCL      9,894
64       France            BLM        NaN
65       France            MAF        NaN
74       France            GLP       8934
117      France            MTQ      11278
120      France            MYT       3300
150      France            REU       2230

     País/Região Código do País PIB 2019
7    Netherlands            ABW    2,903
130  Netherlands            CUW    3,195
131  Netherlands            SXM    1,059
132  Netherlands            NLD  902,355

 

In [16]:
# Remove linhas de 'dados_extras_df' que já existem no DataFrame 'temp2_df'

# Hong Kong e Macau   
dados_extras_df = dados_extras_df.drop([33, 111])

# Ilhas Faroe e Groelândia   
dados_extras_df = dados_extras_df.drop([57, 73])

# Territórios pertencentes à França   
dados_extras_df = dados_extras_df.drop([61, 62, 63, 64, 65, 74, 117, 120, 150])

# Territórios pertencentes à Holanda  
dados_extras_df = dados_extras_df.drop([7, 130, 131])

# Territórios pertencentes aos Estados Unidos   
dados_extras_df = dados_extras_df.drop([75, 147])

In [17]:
# Exibe uma listagem com os países que devem se tratados
# Essas informações apresentam duplicações, pois, já existem no DataFrame 'temp2_df'
# Repare que as anomalias foram corrigidas
paises = ['China', 'Denmark', 'France', 'Netherlands', 'United States']
cols = ['País/Região', 'Código do País', 'PIB 2019']

for pais in paises:
    print(dados_extras_df[dados_extras_df['País/Região'] == pais][cols])
    print()

   País/Região Código do País    PIB 2019
34       China            CHN  14,140,163

   País/Região Código do País PIB 2019
44     Denmark            DNK  347,176

   País/Região Código do País   PIB 2019
60      France            FRA  2,707,074

     País/Região Código do País PIB 2019
132  Netherlands            NLD  902,355

       País/Região Código do País    PIB 2019
182  United States            USA  21,439,453



In [18]:
# Remove a vírgula dos valores da coluna PIB 2019
dados_extras_df = dados_extras_df.replace({'PIB 2019': ','}, {'PIB 2019': ''}, regex=True)

In [19]:
# Dicionário com o tipo de dados de algumas colunas do DataFrame
tipos_dados = {
    'Código do País': str, 
    'País/Região': str,
    'PIB 2019': float
} 

# Converte os tipos de dados de algumas das colunas do DataFrame
dados_extras_df = dados_extras_df.astype(tipos_dados)

In [20]:
# Define uma nova ordem para as colunas do DataFrame 'dados_extras_df'
cols = ['Código do País', 'País/Região', 'Data do Primeiro Caso', 'PIB 2019', 'Taxa de Fumantes', 
        'Taxa de Mortalidade por Doenças Pulmonares', 'Total de Leitos Hospitalares', 'Temperatura Média (Janeiro - Março)',
        'Taxa de Umidade (Janeiro - Março)']

# Altera o DataFrame
dados_extras_df = dados_extras_df[cols]

# 6. Merge dos DataFrames: *temp2_df* e *dados_extras_df*

Nessa etapa, realizamos o merge entre os dois últimos DataFrames que foram gerados. Em seguida, criamos o arquivo CSV final, o qual será utilizado para a aplicação das técnicas de Aprendizado de Máquina.

In [21]:
# Exibe o cabeçalho de 'temp2_df'
temp2_df.head()

,Província/Estado,País/Região,Latitude,Longitude,Data,Casos Confirmados,Fatalidades,População,Densidade (Km/2),Área (Km/2),Taxa de Fertilidade,Média de Idade,População Urbana
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0,0,38928346.0,60.0,652860.0,4.6,18,25 %
1,NaN,Albania,41.1533,20.1683,1/22/20,0,0,2877797.0,105.0,27400.0,1.6,36,63 %
2,NaN,Algeria,28.0339,1.6596,1/22/20,0,0,43851044.0,18.0,2381740.0,3.1,29,73 %
3,NaN,Andorra,42.5063,1.5218,1/22/20,0,0,77265.0,164.0,470.0,N.A.,N.A.,88 %
4,NaN,Angola,-11.2027,17.8739,1/22/20,0,0,32866272.0,26.0,1246700.0,5.6,17,67 %


In [22]:
# Exibe o cabeçalho de 'dados_extras_df'
dados_extras_df.head()

,Código do País,País/Região,Data do Primeiro Caso,PIB 2019,Taxa de Fumantes,Taxa de Mortalidade por Doenças Pulmonares,Total de Leitos Hospitalares,Temperatura Média (Janeiro - Março),Taxa de Umidade (Janeiro - Março)
0,AFG,Afghanistan,2/23/2020,18734.0,NaN,37.62,0.5,6.216667,56.933333
1,ALB,Albania,3/7/2020,15418.0,29.4,11.67,2.9,13.950000,69.900000
2,DZA,Algeria,2/24/2020,172781.0,NaN,8.77,1.9,19.566667,61.716667
3,AND,Andorra,3/1/2020,91527.0,32.5,NaN,2.5,7.383333,75.233333
4,ATG,Antigua and Barbuda,3/12/2020,1688.0,NaN,11.76,3.8,NaN,NaN


In [23]:
# Realiza o merge entre 'temp2_df' e 'dados_extras_df'                      
arquivo_final_df = pd.merge(temp2_df, dados_extras_df, how ='left', on='País/Região')

In [24]:
# Exibe o cabeçalho de 'arquivo_final_df'
arquivo_final_df.head()

,Província/Estado,País/Região,Latitude,Longitude,Data,Casos Confirmados,Fatalidades,População,Densidade (Km/2),Área (Km/2),...,Média de Idade,População Urbana,Código do País,Data do Primeiro Caso,PIB 2019,Taxa de Fumantes,Taxa de Mortalidade por Doenças Pulmonares,Total de Leitos Hospitalares,Temperatura Média (Janeiro - Março),Taxa de Umidade (Janeiro - Março)
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0,0,38928346.0,60.0,652860.0,...,18,25 %,AFG,2/23/2020,18734.0,NaN,37.62,0.5,6.216667,56.933333
1,NaN,Albania,41.1533,20.1683,1/22/20,0,0,2877797.0,105.0,27400.0,...,36,63 %,ALB,3/7/2020,15418.0,29.4,11.67,2.9,13.950000,69.900000
2,NaN,Algeria,28.0339,1.6596,1/22/20,0,0,43851044.0,18.0,2381740.0,...,29,73 %,DZA,2/24/2020,172781.0,NaN,8.77,1.9,19.566667,61.716667
3,NaN,Andorra,42.5063,1.5218,1/22/20,0,0,77265.0,164.0,470.0,...,N.A.,88 %,AND,3/1/2020,91527.0,32.5,NaN,2.5,7.383333,75.233333
4,NaN,Angola,-11.2027,17.8739,1/22/20,0,0,32866272.0,26.0,1246700.0,...,17,67 %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Exibe as dimensões do DataFrame final
print("Dimensões do arquivo final:\n")
print(f"Total de linhas: {arquivo_final_df.shape[0]}")
print(f"Total de colunas: {arquivo_final_df.shape[1]}")

Dimensões do arquivo final:

Total de linhas: 22355
Total de colunas: 21


In [26]:
# Exibe os tipos de dados do DataFrame
arquivo_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22355 entries, 0 to 22354
Data columns (total 21 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Província/Estado                            6885 non-null   object 
 1   País/Região                                 22355 non-null  object 
 2   Latitude                                    22355 non-null  float64
 3   Longitude                                   22355 non-null  float64
 4   Data                                        22355 non-null  object 
 5   Casos Confirmados                           22355 non-null  int64  
 6   Fatalidades                                 22355 non-null  int64  
 7   População                                   21335 non-null  float64
 8   Densidade (Km/2)                            21335 non-null  float64
 9   Área (Km/2)                                 21335 non-null  float64
 10  Taxa de Fe

In [27]:
# Remove o sinal de '%', da coluna População Urbana
arquivo_final_df = arquivo_final_df.replace({'População Urbana': '%'}, {'População Urbana': ''}, regex=True)

In [28]:
# Troca todos os N.A. valores por NaN
arquivo_final_df = arquivo_final_df.replace(to_replace =['N.A.'], value = np.nan)

In [29]:
# Corrige os tipos de dados do DataFrame 'arquivo_final_df'

# Dicionário com o tipo de dados de algumas colunas do DataFrame
tipos_dados = { 
    'Província/Estado': str,
    'País/Região': str,
    'Latitude': float,
    'Longitude': float,
    #'Data': ,
    'Casos Confirmados': int,
    'Fatalidades': int,
    'População': float,
    'Densidade (Km/2)': float,
    'Área (Km/2)': float,
    'Taxa de Fertilidade': float,
    'Média de Idade': float,
    'População Urbana': float,
    'Código do País': str,
    #'Data do Primeiro Caso': ,
    'PIB 2019': float,
    'Taxa de Fumantes': float,
    'Taxa de Mortalidade por Doenças Pulmonares': float,
    'Total de Leitos Hospitalares': float,
    'Temperatura Média (Janeiro - Março)': float,
    'Taxa de Umidade (Janeiro - Março)': float
} 

# Converte os tipos de dados de algumas das colunas do DataFrame
arquivo_final_df = arquivo_final_df.astype(tipos_dados)

In [30]:
# Troca todos os nan valores por NaN
arquivo_final_df = arquivo_final_df.replace(to_replace =['nan'], value = np.nan)

In [31]:
# Exibe informações do DataFrame 'arquivo_final_df'
arquivo_final_df.head()

,Província/Estado,País/Região,Latitude,Longitude,Data,Casos Confirmados,Fatalidades,População,Densidade (Km/2),Área (Km/2),...,Média de Idade,População Urbana,Código do País,Data do Primeiro Caso,PIB 2019,Taxa de Fumantes,Taxa de Mortalidade por Doenças Pulmonares,Total de Leitos Hospitalares,Temperatura Média (Janeiro - Março),Taxa de Umidade (Janeiro - Março)
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0,0,38928346.0,60.0,652860.0,...,18.0,25.0,AFG,2/23/2020,18734.0,NaN,37.62,0.5,6.216667,56.933333
1,NaN,Albania,41.1533,20.1683,1/22/20,0,0,2877797.0,105.0,27400.0,...,36.0,63.0,ALB,3/7/2020,15418.0,29.4,11.67,2.9,13.950000,69.900000
2,NaN,Algeria,28.0339,1.6596,1/22/20,0,0,43851044.0,18.0,2381740.0,...,29.0,73.0,DZA,2/24/2020,172781.0,NaN,8.77,1.9,19.566667,61.716667
3,NaN,Andorra,42.5063,1.5218,1/22/20,0,0,77265.0,164.0,470.0,...,NaN,88.0,AND,3/1/2020,91527.0,32.5,NaN,2.5,7.383333,75.233333
4,NaN,Angola,-11.2027,17.8739,1/22/20,0,0,32866272.0,26.0,1246700.0,...,17.0,67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Exibe os tipos de dados do DataFrame
arquivo_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22355 entries, 0 to 22354
Data columns (total 21 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Província/Estado                            6885 non-null   object 
 1   País/Região                                 22355 non-null  object 
 2   Latitude                                    22355 non-null  float64
 3   Longitude                                   22355 non-null  float64
 4   Data                                        22355 non-null  object 
 5   Casos Confirmados                           22355 non-null  int64  
 6   Fatalidades                                 22355 non-null  int64  
 7   População                                   21335 non-null  float64
 8   Densidade (Km/2)                            21335 non-null  float64
 9   Área (Km/2)                                 21335 non-null  float64
 10  Taxa de Fe

In [33]:
# Define uma nova ordem para as colunas do DataFrame
cols = ['Código do País', 'País/Região', 'Província/Estado', 'Latitude', 'Longitude', 'PIB 2019', 'População', 
        'População Urbana', 'Área (Km/2)', 'Densidade (Km/2)', 'Taxa de Fertilidade', 'Média de Idade', 'Taxa de Fumantes',
        'Taxa de Mortalidade por Doenças Pulmonares', 'Total de Leitos Hospitalares', 'Temperatura Média (Janeiro - Março)', 
        'Taxa de Umidade (Janeiro - Março)', 'Data do Primeiro Caso', 'Data', 'Casos Confirmados', 'Fatalidades']

# Atualiza as colunas do novo DataFrame
arquivo_final_df = arquivo_final_df[cols]

In [34]:
# Exibe informações do DataFrame 'arquivo_final_df'
arquivo_final_df.head()

,Código do País,País/Região,Província/Estado,Latitude,Longitude,PIB 2019,População,População Urbana,Área (Km/2),Densidade (Km/2),...,Média de Idade,Taxa de Fumantes,Taxa de Mortalidade por Doenças Pulmonares,Total de Leitos Hospitalares,Temperatura Média (Janeiro - Março),Taxa de Umidade (Janeiro - Março),Data do Primeiro Caso,Data,Casos Confirmados,Fatalidades
0,AFG,Afghanistan,NaN,33.0000,65.0000,18734.0,38928346.0,25.0,652860.0,60.0,...,18.0,NaN,37.62,0.5,6.216667,56.933333,2/23/2020,1/22/20,0,0
1,ALB,Albania,NaN,41.1533,20.1683,15418.0,2877797.0,63.0,27400.0,105.0,...,36.0,29.4,11.67,2.9,13.950000,69.900000,3/7/2020,1/22/20,0,0
2,DZA,Algeria,NaN,28.0339,1.6596,172781.0,43851044.0,73.0,2381740.0,18.0,...,29.0,NaN,8.77,1.9,19.566667,61.716667,2/24/2020,1/22/20,0,0
3,AND,Andorra,NaN,42.5063,1.5218,91527.0,77265.0,88.0,470.0,164.0,...,NaN,32.5,NaN,2.5,7.383333,75.233333,3/1/2020,1/22/20,0,0
4,NaN,Angola,NaN,-11.2027,17.8739,NaN,32866272.0,67.0,1246700.0,26.0,...,17.0,NaN,NaN,NaN,NaN,NaN,NaN,1/22/20,0,0


In [35]:
# Exibe informações de um país específico -> Brazil
arquivo_final_df[arquivo_final_df['Código do País'] == 'BRA']

,Código do País,País/Região,Província/Estado,Latitude,Longitude,PIB 2019,População,População Urbana,Área (Km/2),Densidade (Km/2),...,Média de Idade,Taxa de Fumantes,Taxa de Mortalidade por Doenças Pulmonares,Total de Leitos Hospitalares,Temperatura Média (Janeiro - Março),Taxa de Umidade (Janeiro - Março),Data do Primeiro Caso,Data,Casos Confirmados,Fatalidades
28,BRA,Brazil,NaN,-14.235,-51.9253,1847020.0,212559417.0,88.0,8358140.0,25.0,...,33.0,15.3,26.57,2.2,25.816667,81.683333,2/24/2020,1/22/20,0,0
291,BRA,Brazil,NaN,-14.235,-51.9253,1847020.0,212559417.0,88.0,8358140.0,25.0,...,33.0,15.3,26.57,2.2,25.816667,81.683333,2/24/2020,1/23/20,0,0
554,BRA,Brazil,NaN,-14.235,-51.9253,1847020.0,212559417.0,88.0,8358140.0,25.0,...,33.0,15.3,26.57,2.2,25.816667,81.683333,2/24/2020,1/24/20,0,0
817,BRA,Brazil,NaN,-14.235,-51.9253,1847020.0,212559417.0,88.0,8358140.0,25.0,...,33.0,15.3,26.57,2.2,25.816667,81.683333,2/24/2020,1/25/20,0,0
1080,BRA,Brazil,NaN,-14.235,-51.9253,1847020.0,212559417.0,88.0,8358140.0,25.0,...,33.0,15.3,26.57,2.2,25.816667,81.683333,2/24/2020,1/26/20,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21068,BRA,Brazil,NaN,-14.235,-51.9253,1847020.0,212559417.0,88.0,8358140.0,25.0,...,33.0,15.3,26.57,2.2,25.816667,81.683333,2/24/2020,4/11/20,20727,1124
21331,BRA,Brazil,NaN,-14.235,-51.9253,1847020.0,212559417.0,88.0,8358140.0,25.0,...,33.0,15.3,26.57,2.2,25.816667,81.683333,2/24/2020,4/12/20,22192,1223
21594,BRA,Brazil,NaN,-14.235,-51.9253,1847020.0,212559417.0,88.0,8358140.0,25.0,...,33.0,15.3,26.57,2.2,25.816667,81.683333,2/24/2020,4/13/20,23430,1328
21857,BRA,Brazil,NaN,-14.235,-51.9253,1847020.0,212559417.0,88.0,8358140.0,25.0,...,33.0,15.3,26.57,2.2,25.816667,81.683333,2/24/2020,4/14/20,25262,1532


# 7. Corrige mais anomalias

Corrige mais algumas anomalias existentes no arquivo final.

In [36]:
# Exibe informações de um país específico -> Canada e Princess Diamond
# Observe que os casos confirmados estão negativos (?)
arquivo_final_df[(arquivo_final_df['País/Região'] == 'Canada') & 
                 (arquivo_final_df['Província/Estado'] == 'Diamond Princess')]

,Código do País,País/Região,Província/Estado,Latitude,Longitude,PIB 2019,População,População Urbana,Área (Km/2),Densidade (Km/2),...,Média de Idade,Taxa de Fumantes,Taxa de Mortalidade por Doenças Pulmonares,Total de Leitos Hospitalares,Temperatura Média (Janeiro - Março),Taxa de Umidade (Janeiro - Março),Data do Primeiro Caso,Data,Casos Confirmados,Fatalidades
231,CAN,Canada,Diamond Princess,0.0,0.0,1730914.0,37742154.0,81.0,9093510.0,4.0,...,41.0,14.95,19.01,2.7,-1.866667,88.583333,1/24/2020,1/22/20,0,0
494,CAN,Canada,Diamond Princess,0.0,0.0,1730914.0,37742154.0,81.0,9093510.0,4.0,...,41.0,14.95,19.01,2.7,-1.866667,88.583333,1/24/2020,1/23/20,0,0
757,CAN,Canada,Diamond Princess,0.0,0.0,1730914.0,37742154.0,81.0,9093510.0,4.0,...,41.0,14.95,19.01,2.7,-1.866667,88.583333,1/24/2020,1/24/20,0,0
1020,CAN,Canada,Diamond Princess,0.0,0.0,1730914.0,37742154.0,81.0,9093510.0,4.0,...,41.0,14.95,19.01,2.7,-1.866667,88.583333,1/24/2020,1/25/20,0,0
1283,CAN,Canada,Diamond Princess,0.0,0.0,1730914.0,37742154.0,81.0,9093510.0,4.0,...,41.0,14.95,19.01,2.7,-1.866667,88.583333,1/24/2020,1/26/20,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21271,CAN,Canada,Diamond Princess,0.0,0.0,1730914.0,37742154.0,81.0,9093510.0,4.0,...,41.0,14.95,19.01,2.7,-1.866667,88.583333,1/24/2020,4/11/20,0,1
21534,CAN,Canada,Diamond Princess,0.0,0.0,1730914.0,37742154.0,81.0,9093510.0,4.0,...,41.0,14.95,19.01,2.7,-1.866667,88.583333,1/24/2020,4/12/20,-1,1
21797,CAN,Canada,Diamond Princess,0.0,0.0,1730914.0,37742154.0,81.0,9093510.0,4.0,...,41.0,14.95,19.01,2.7,-1.866667,88.583333,1/24/2020,4/13/20,-1,-1
22060,CAN,Canada,Diamond Princess,0.0,0.0,1730914.0,37742154.0,81.0,9093510.0,4.0,...,41.0,14.95,19.01,2.7,-1.866667,88.583333,1/24/2020,4/14/20,-1,-1


In [37]:
# Remove as linhas onde o terrirório do Canadá seja o 'Diamond Princess'
arquivo_final_df = arquivo_final_df.drop(arquivo_final_df[(arquivo_final_df['País/Região'] == 'Canada') & 
                                                          (arquivo_final_df['Província/Estado'] == 'Diamond Princess')].index)

In [38]:
# Verifica se ainda existe alguma linha onde o valor de 'Casos Confirmados' ou 'Fatalidades' seja negtivo
arquivo_final_df[(arquivo_final_df['Casos Confirmados'] < 0) | (arquivo_final_df['Fatalidades'] < 0)].count()

Código do País                                0
País/Região                                   0
Província/Estado                              0
Latitude                                      0
Longitude                                     0
PIB 2019                                      0
População                                     0
População Urbana                              0
Área (Km/2)                                   0
Densidade (Km/2)                              0
Taxa de Fertilidade                           0
Média de Idade                                0
Taxa de Fumantes                              0
Taxa de Mortalidade por Doenças Pulmonares    0
Total de Leitos Hospitalares                  0
Temperatura Média (Janeiro - Março)           0
Taxa de Umidade (Janeiro - Março)             0
Data do Primeiro Caso                         0
Data                                          0
Casos Confirmados                             0
Fatalidades                             

In [39]:
# Exibe as dimensões do DataFrame final
print("Dimensões do arquivo final:\n")
print(f"Total de linhas: {arquivo_final_df.shape[0]}")
print(f"Total de colunas: {arquivo_final_df.shape[1]}")

Dimensões do arquivo final:

Total de linhas: 22270
Total de colunas: 21


In [40]:
# Lista com o nome dos navios de cruzeiro em quarentena
navios = ['Diamond Princess', 'MS Zaandam', 'MS Westerdam', 'World Dream', 'Costa Serena', 'Costa Victoria', 'Norwegian Jewel', 
          'Grand Princess', 'A Sara', 'Costa Magica', 'Braemar', 'Costa Luminosa', 'Silver Explorer', 'Silver Shadow', 
          'Ruby Princess', 'Ovation of the Seas', 'Voyager of the Seas', 'Artania', 'Costa Favolosa', 'Horizon', 
          'Oasis of the Seas', 'Liberty of the Seas', 'Coral Princess', 'Celebrity Solstice', 'Disney Wonder', 'Greg Mortimer',
          'Pride of America', 'Celebrity Flora', 'MS Rotterdam', 'River Anuket']

# Habilita/Desabilita a opção par exibir todas as linhas
pd.set_option('display.max_rows', False)

# Exibe as linhas onde o 'País/Região' seja algum navio de cruzeiro
# No caso, essas linhas serão mantidas no DataFrame
arquivo_final_df[arquivo_final_df['País/Região'].isin(navios)]

,Código do País,País/Região,Província/Estado,Latitude,Longitude,PIB 2019,População,População Urbana,Área (Km/2),Densidade (Km/2),...,Média de Idade,Taxa de Fumantes,Taxa de Mortalidade por Doenças Pulmonares,Total de Leitos Hospitalares,Temperatura Média (Janeiro - Março),Taxa de Umidade (Janeiro - Março),Data do Primeiro Caso,Data,Casos Confirmados,Fatalidades
88,NaN,Diamond Princess,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/22/20,0,0
251,NaN,MS Zaandam,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/22/20,0,0
351,NaN,Diamond Princess,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/23/20,0,0
514,NaN,MS Zaandam,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/23/20,0,0
614,NaN,Diamond Princess,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/24/20,0,0
777,NaN,MS Zaandam,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/24/20,0,0
877,NaN,Diamond Princess,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/25/20,0,0
1040,NaN,MS Zaandam,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/25/20,0,0
1140,NaN,Diamond Princess,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1/26/20,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# 8. Exporta a nova base de dados

Exporta o DataFrame, gerando o arquivo **covid19.csv**. Esse arquivo será utilizado para aplicação das técnicas de Aprendizado de Máquina. Observe que os dados de dois navios de cruzeiro, Diamond Princess e MS Zaandam, também foram gravados.

In [42]:
# Exporta o DataFrame -> 'covid19.csv'
arquivo_final_df.to_csv('data/artigo/covid19.csv', index=False)